# Simulation template.

In this notebook we produce the results of the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 656 ms, sys: 40.3 ms, total: 696 ms
Wall time: 694 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = "/home/linneu/ssh_csvs/ssh1/periodic_100_6561.csv"   
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = "/home/linneu/ssh_simulations/ssh1/periodic_100_6561"
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 6.2 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/linneu/ssh_csvs/ssh2/periodic_140_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/linneu/ssh_simulations/ssh2/periodic_140_6561"
random_state = 729


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 918540 entries, 0 to 918539
Columns: 148 entries, id to feat139
dtypes: float64(143), int32(3), object(2)
memory usage: 1.0+ GB
CPU times: user 15.9 s, sys: 1.04 s, total: 17 s
Wall time: 17.3 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130839,-0.300829,...,-0.080270,0.115548,-0.174799,-0.244861,0.319462,0.059061,-0.000568,0.244879,-0.300829,-0.130839
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.130839,-0.300829,...,0.080270,0.115548,0.174799,-0.244861,-0.319462,0.059061,0.000568,0.244879,0.300829,-0.130839
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.136032,0.040156,...,0.174723,0.086763,-0.070199,0.105493,-0.137161,-0.181862,0.180933,0.052357,-0.040156,0.136032
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.016772,0.038115,...,0.011447,-0.002690,0.015595,0.024714,-0.039364,-0.009687,0.001017,-0.028996,0.038115,0.016772
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.096087,0.024817,...,0.105770,0.025590,-0.024172,0.086413,-0.100604,-0.112297,0.113588,0.022113,-0.024817,0.096087


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  140
n_hamiltonians:  6561
n_ts:  2
CPU times: user 403 µs, sys: 27 µs, total: 430 µs
Wall time: 278 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.8414875781130925
% val:  0.0
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  5521
number of val hamiltonians:  0
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [00:49<1:22:29, 50.00s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [01:39<1:21:10, 49.70s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [1:20:55<00:00, 48.43s/it]

running experiments: 100%|██████████| 100/100 [1:20:55<00:00, 48.55s/it]

CPU times: user 17h 34min 4s, sys: 1min 4s, total: 17h 35min 9s
Wall time: 1h 20min 55s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.130839,-0.300829,...,-0.080270,0.115548,-0.174799,-0.244861,0.319462,0.059061,-0.000568,0.244879,-0.300829,-0.130839
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,0.130839,-0.300829,...,0.080270,0.115548,0.174799,-0.244861,-0.319462,0.059061,0.000568,0.244879,0.300829,-0.130839
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,-0.136032,0.040156,...,0.174723,0.086763,-0.070199,0.105493,-0.137161,-0.181862,0.180933,0.052357,-0.040156,0.136032
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,0.016772,0.038115,...,0.011447,-0.002690,0.015595,0.024714,-0.039364,-0.009687,0.001017,-0.028996,0.038115,0.016772
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.096087,0.024817,...,0.105770,0.025590,-0.024172,0.086413,-0.100604,-0.112297,0.113588,0.022113,-0.024817,0.096087
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.037409,0.017407,...,0.064699,0.087166,-0.069474,-0.006214,-0.025629,-0.076251,0.078393,0.045301,-0.017407,0.037409
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.001081,0.008635,...,-0.052698,-0.054268,0.035200,-0.005245,0.001839,0.033479,-0.012524,-0.019661,0.008635,-0.001081
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.037034,-0.082887,...,-0.029095,-0.011589,-0.023527,-0.045985,0.084698,0.025950,-0.004245,0.059527,-0.082887,-0.037034
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.105296,0.019814,...,0.076874,-0.034699,0.019704,0.110168,-0.114864,-0.090059,0.096749,-0.003643,-0.019814,0.105296
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.045950,0.023837,...,0.071514,0.118769,-0.103239,-0.027565,-0.023510,-0.100008,0.109383,0.069605,-0.023837,0.045950


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 164 ms, sys: 4 ms, total: 168 ms
Wall time: 168 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
415109,2965,H_-0.2_0.45_-0.000500575.dat,-0.20,0.45,-0.000501,0,0,train,0.054049,-0.056144,...,-0.059056,0.096694,-0.009522,0.055974,0.044456,0.001007,0.052799,-0.037397,0.056144,-0.054049
551856,3941,H_0.4_0.65_2.00303.dat,0.40,0.65,2.003033,2,2,train,-0.001557,-0.011595,...,0.020088,-0.046326,-0.026113,0.041161,0.018498,-0.022641,0.000584,0.010087,-0.011595,-0.001557
373806,2670,H_-0.4_1.9_2.00113.dat,-0.40,1.90,2.001132,2,2,val,0.048542,-0.033691,...,-0.093336,-0.068942,-0.090062,-0.084020,-0.087631,-0.099892,-0.016997,-0.082964,0.033691,-0.048542
724920,5178,H_1.15_1.75_2.0066.dat,1.15,1.75,2.006598,2,2,val,0.243800,-0.244725,...,0.148992,-0.127773,0.174477,-0.172865,0.287957,-0.189379,0.204035,-0.213293,0.244725,-0.243800
318545,2275,H_-0.6_-1.65_2.00506.dat,-0.60,-1.65,2.005062,2,2,train,-0.050850,-0.055151,...,-0.022068,-0.092133,0.123066,0.109570,0.048922,0.082906,-0.035537,-0.056511,-0.055151,-0.050850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790560,5646,H_1.45_0.85_-1.00389.dat,1.45,0.85,-1.003887,-1,-1,val,0.037909,0.041245,...,0.059760,-0.060264,0.100675,0.017513,-0.120788,0.000176,-0.021250,-0.004572,0.041245,0.037909
422066,3014,H_-0.15_-1.15_1.00374.dat,-0.15,-1.15,1.003742,1,1,train,0.066646,-0.054147,...,-0.065942,0.030247,-0.020040,-0.028967,0.037457,-0.073089,0.073854,-0.112281,0.054147,-0.066646
213580,1525,H_-1.1_1.35_0.000265618.dat,-1.10,1.35,0.000266,0,0,val,-0.195313,-0.052576,...,-0.068887,0.019407,-0.022566,0.061578,0.017819,-0.102582,0.096288,0.013000,0.052576,0.195313
671432,4795,H_0.95_-1.2_1.00124.dat,0.95,-1.20,1.001236,1,1,val,0.013526,0.020851,...,0.084756,-0.067157,0.064782,-0.045677,0.056985,-0.015852,0.009700,-0.004741,0.020851,0.013526


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.4208199969516842
% val:  0.42066758116140834
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  2761
number of val hamiltonians:  2760
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [2667, 1746, 5681, 6551, 6102, 1859, 4234, 1568, 1697, 4480, 2905, 3843, 2351, 3128, 3027, 6028, 5805, 3324, 4392, 115, 6158, 5935, 1935, 1938, 5105, 4782, 4056, 263, 6348, 4697, 921, 4328, 4468, 4781, 6368, 3976, 864, 3955, 4311, 764, 2695, 2240, 3487, 1983, 2276, 1829, 714, 2469, 3183, 3121, 4094, 5009, 5807, 3421, 5378, 5515, 3145, 3203, 3344, 3869, 55, 1345, 3727, 2758, 4168, 3053, 6545, 1408, 5467, 4659, 946, 2755, 3112, 5215, 3619, 2651, 3564, 2121, 1589, 867, 1504, 1868, 3499, 3568, 3841, 4626, 3286, 3808, 1658, 2109, 69, 2553, 1695, 2169, 5808, 5565, 2697, 3882, 5182, 5363, 1783, 6167, 2191, 3408, 4582, 1210, 385, 4102, 262, 953, 4722, 189, 5463, 214, 53, 1729, 6324, 2315, 5056, 2647, 5850, 5185, 3111, 6056, 555,

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 240 µs, sys: 1e+03 ns, total: 241 µs
Wall time: 138 µs


,t1,t2,type_of,-1,0,1,2,phase,pred_phase
id,,,,,,,,,
0,-2.0,-2.00,test,0.314286,0.300000,0.300000,0.085714,999,-1
1,-2.0,-1.95,test,0.385714,0.307143,0.285714,0.021429,-1,-1
2,-2.0,-1.90,test,0.364286,0.321429,0.264286,0.050000,-1,-1
3,-2.0,-1.85,test,0.357143,0.335714,0.250000,0.057143,-1,-1
4,-2.0,-1.80,test,0.371429,0.307143,0.257143,0.064286,-1,-1
...,...,...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.492857,0.178571,0.114286,0.214286,-1,-1
6557,2.0,1.85,test,0.450000,0.228571,0.085714,0.235714,-1,-1
6558,2.0,1.90,test,0.392857,0.250000,0.107143,0.250000,-1,-1


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  918540


,id,phase,pred_phase,type_of
0,0,999,-1,test
1,0,999,-1,test
2,0,999,2,test
3,0,999,-1,test
4,0,999,1,test
...,...,...,...,...
918535,6560,999,-1,test
918536,6560,999,-1,test
918537,6560,999,-1,test
918538,6560,999,0,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9997852744864697,
 'eigenvector_val': 0.9640087991718427,
 'eigenvector_test': 0.6491352070513886,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 0.9974637681159421,
 'hamiltonian_test': 0.9022118742724098}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}